Code from https://dacon.io/competitions/official/235877/codeshare/4712?page=1&dtype=recent

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import os

In [3]:
DATA_DIR = "/content/drive/MyDrive/Colab Notebooks/Dacon/abalone/data"
train = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'))
test = pd.read_csv(os.path.join(DATA_DIR, 'test.csv'))

In [4]:
submission = pd.read_csv(os.path.join(DATA_DIR, 'sample_submission.csv'))

In [5]:
def check_missing_value(df):
  missing_values = df.isnull().sum().sort_values(ascending=False)
  missing_percentage = (missing_values / len(df)) * 100
  result = pd.concat([missing_values, missing_percentage], axis=1, keys=['Missing values', '% Missing'])
  display(result)
check_missing_value(train)

,Missing values,% Missing
id,0,0.0
Gender,0,0.0
Lenght,0,0.0
Diameter,0,0.0
Height,0,0.0
Whole Weight,0,0.0
Shucked Weight,0,0.0
Viscra Weight,0,0.0
Shell Weight,0,0.0
Target,0,0.0


In [6]:
from sklearn.preprocessing import LabelEncoder

In [7]:
encoder = LabelEncoder()
train["Gender"] = encoder.fit_transform(train['Gender'])

In [8]:
test["Gender"] = encoder.transform(test['Gender'])

In [10]:
keys = encoder.classes_
values = encoder.transform(keys)
labels = dict(zip(keys, values))
labels

{'F': 0, 'I': 1, 'M': 2}

In [18]:
train_X = train.drop(['id', 'Target'], axis=1)
train_y = train["Target"]

In [14]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor

models = []

models.append(RandomForestRegressor(random_state=32))
models.append(LinearRegression())
models.append(GradientBoostingRegressor(learning_rate=0.06, criterion='mae', random_state=32))

In [19]:
for model in models:
  model.fit(train_X, train_y)

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:1859: FutureWarning: criterion='mae' was deprecated in version 0.24 and will be removed in version 1.1 (renaming of 0.26). The correct way of minimizing the absolute error is to use  loss='absolute_error' instead.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py:370: FutureWarning: Criterion 'mae' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='absolute_error'` which is equivalent.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py:370: FutureWarning: Criterion 'mae' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='absolute_error'` which is equivalent.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py:370: FutureWarning: Criterion 'mae' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='absolute_error'` which is equivalent.
  FutureWarning,
/usr/local/lib/p

In [17]:
test_X = test.drop(['id'], axis=1)

In [20]:
prediction = None

In [21]:
for model in models:
  if prediction is None:
    prediction = model.predict(test_X)
  else:
    prediction += model.predict(test_X)

  
prediction /= len(models)

In [22]:
prediction[:5]

array([ 9.60423313, 12.72947467,  5.66079783, 11.19351372,  9.9743309 ])

In [24]:
prediction = prediction.astype(int)

In [25]:
prediction[:5]

array([ 9, 12,  5, 11,  9])

In [27]:
submission['Target'] = prediction

In [29]:
submission.head()

,id,Target
0,1,9
1,2,12
2,3,5
3,4,11
4,5,9


In [30]:
submission.to_csv('submit.csv', index=False)